In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import re

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


In [3]:
df18 = pd.read_excel('MUNIC//Base_MUNIC_2018.xlsx', sheet_name='Saúde', usecols=['Cod Municipio','MSAU01','MSAU03','MSAU04','MSAU05','MSAU06'])
df18

,Cod Municipio,MSAU01,MSAU03,MSAU04,MSAU05,MSAU06
0,1100015,Secretaria exclusiva,Masculino,46,Preta,Ensino médio (2º Grau) completo
1,1100023,Secretaria exclusiva,Masculino,37,Branca,Especialização
2,1100031,Secretaria exclusiva,Masculino,45,Parda,Ensino superior completo
3,1100049,Secretaria exclusiva,Feminino,45,Branca,Especialização
4,1100056,Secretaria exclusiva,Feminino,62,Branca,Ensino médio (2º Grau) completo
...,...,...,...,...,...,...
5565,5222005,Secretaria exclusiva,Feminino,31,Parda,Especialização
5566,5222054,Secretaria exclusiva,Masculino,58,Branca,Ensino médio (2º Grau) completo
5567,5222203,Secretaria exclusiva,Masculino,37,Parda,Ensino superior completo
5568,5222302,Secretaria exclusiva,Feminino,32,Branca,Ensino superior incompleto


In [40]:
df21 = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name='Saúde', usecols=['CodMun','Msau01','Msau03','Msau04','Msau05','Msau06'])
df21

,CodMun,Msau01,Msau03,Msau04,Msau05,Msau06
0,1100015,Secretaria exclusiva,Masculino,37,Parda,Especialização
1,1100023,Secretaria exclusiva,Feminino,44,Branca,Especialização
2,1100031,Secretaria exclusiva,Masculino,49,Parda,Ensino superior completo
3,1100049,Secretaria exclusiva,Feminino,42,Parda,Ensino superior completo
4,1100056,Secretaria exclusiva,Masculino,35,Branca,Ensino superior completo
...,...,...,...,...,...,...
5565,5222005,Secretaria exclusiva,Feminino,32,Parda,Ensino superior completo
5566,5222054,Secretaria exclusiva,Masculino,61,Parda,Ensino médio (2º Grau) completo
5567,5222203,Secretaria exclusiva,Feminino,45,Preta,Ensino médio (2º Grau) incompleto
5568,5222302,Secretaria exclusiva,Feminino,39,Branca,Ensino superior completo


In [41]:
uf = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name = 'Variáveis externas', usecols=[0,1,2,3])
uf

,CodMun,UF,Cod UF,Mun
0,1100015,RO,11,Alta Floresta DOeste
1,1100023,RO,11,Ariquemes
2,1100031,RO,11,Cabixi
3,1100049,RO,11,Cacoal
4,1100056,RO,11,Cerejeiras
...,...,...,...,...
5565,5222005,GO,52,Vianópolis
5566,5222054,GO,52,Vicentinópolis
5567,5222203,GO,52,Vila Boa
5568,5222302,GO,52,Vila Propício


In [42]:
df18= df18.rename(columns={'Cod Municipio':'id_municipio',
                        'MSAU01':'caracterizacao_orgao_gestor',
                        'MSAU03':'genero',
                        'MSAU04' :'idade',
                        'MSAU05':'cor_raca',
                        'MSAU06':'grau_instrucao'}) 

In [43]:
df21= df21.rename(columns={'CodMun':'id_municipio',
                            'Msau01':'caracterizacao_orgao_gestor',
                            'Msau03':'genero',
                            'Msau04' :'idade',
                            'Msau05':'cor_raca',
                            'Msau06':'grau_instrucao'}) 

In [44]:
##adicionando a coluna de 'ano' em cada um dos dfs
df18['ano']=2018
df21['ano']=2021 

In [45]:
df = pd.concat([df18,df21]) #juntando os dataframes

In [46]:
x= uf.pivot_table(columns=('CodMun', 'UF','Cod UF', 'Mun'), aggfunc='size')


In [47]:
uf = pd.DataFrame(x).reset_index()[['CodMun', 'UF','Cod UF', 'Mun']]

In [48]:
df = df.merge(uf, right_on='CodMun',left_on='id_municipio') #adicionando siglas e nome das UFs e dos municipios


In [49]:
df = df.drop(['CodMun'], axis=1)

In [50]:
df = df.rename(columns={'UF':'sigla_uf',
                        'Cod UF':'cod_uf',
                        'Mun':'nome_municipio'}) #padronizando as colunas

In [51]:
def transformar(nome):
    nome = re.sub(r"\bdiretamente\nà \b", "diretamente a ", nome)
    return nome

In [52]:
df['caracterizacao_orgao_gestor']= df['caracterizacao_orgao_gestor'].apply(transformar) ##padronizando os nomes com erro de escrita


In [53]:
df['caracterizacao_orgao_gestor']=df['caracterizacao_orgao_gestor'].str.title()

In [54]:
#padronizando os dados da coluna 'genero'
df['genero']=np.where(df['genero']=='(**)Sem gestor','Sem titular',df['genero'])
df['genero']=np.where(df['genero']=='(**) Sem gestor','Sem titular',df['genero'])
df['genero']=np.where(df['genero']=='Não informou','Sem dados',df['genero'])
df['genero']=np.where(df['genero']=='-','Sem dados',df['genero'])


#padronizando os dados da coluna 'idade'
df['idade']=np.where(df['idade']=='(*) Não soube informar',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='(**)Sem gestor',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='(**) Sem gestor',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Recusa',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Não informou',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='-',np.nan,df['idade'])



#padronizando os dados da coluna 'cor_raca'

df['cor_raca']=np.where(df['cor_raca']=='(**) Sem gestor','Sem titular',df['cor_raca'])
df['cor_raca']=np.where(df['cor_raca']=='(**)Sem gestor','Sem titular',df['cor_raca'])
df['cor_raca']=np.where(df['cor_raca']=='-','Sem dados',df['cor_raca'])
df['cor_raca']=np.where(df['cor_raca']=='Não informou','Sem dados',df['cor_raca'])

#padronizando os dados da coluna 'escolaridade'

df['grau_instrucao']=np.where(df['grau_instrucao']=='(**)Sem gestor','Sem titular',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='(**) Sem gestor','Sem titular',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Não informou','Sem dados',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='-','Sem dados',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Ensino fundamental (1º Grau) incompleto','Até Ensino Fundamental',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Ensino fundamental (1º Grau) completo','Até Ensino Fundamental',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Ensino médio (2º Grau) incompleto','Até Ensino Médio',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Ensino médio (2º Grau) completo','Até Ensino Médio',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Ensino superior incompleto','Até Ensino Superior Completo',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Ensino superior completo','Até Ensino Superior Completo',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Especialização','Até Pós Graduação ou Mestrado',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Mestrado','Até Pós Graduação ou Mestrado',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Doutorado','Até Doutorado',df['grau_instrucao'])

In [56]:
df.columns

Index(['id_municipio', 'caracterizacao_orgao_gestor', 'genero', 'idade',
       'cor_raca', 'grau_instrucao', 'ano', 'sigla_uf', 'cod_uf',
       'nome_municipio'],
      dtype='object')

In [62]:
df['idade'] =pd.to_numeric(df['idade']) #transformando em numerico 


In [63]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [64]:
df=df[['ano', 'sigla_uf','nome_municipio','id_municipio','caracterizacao_orgao_gestor', 'genero', 'cor_raca', 'grau_instrucao', 'faixa_etaria']] #reorganizando as colunas

In [66]:
df

,ano,sigla_uf,nome_municipio,id_municipio,caracterizacao_orgao_gestor,genero,cor_raca,grau_instrucao,faixa_etaria
0,2018,RO,Alta Floresta DOeste,1100015,Secretaria Exclusiva,Masculino,Preta,Até Ensino Médio,Entre 30-49
1,2021,RO,Alta Floresta DOeste,1100015,Secretaria Exclusiva,Masculino,Parda,Até Pós Graduação ou Mestrado,Entre 30-49
2,2018,RO,Ariquemes,1100023,Secretaria Exclusiva,Masculino,Branca,Até Pós Graduação ou Mestrado,Entre 30-49
3,2021,RO,Ariquemes,1100023,Secretaria Exclusiva,Feminino,Branca,Até Pós Graduação ou Mestrado,Entre 30-49
4,2018,RO,Cabixi,1100031,Secretaria Exclusiva,Masculino,Parda,Até Ensino Superior Completo,Entre 30-49
...,...,...,...,...,...,...,...,...,...
11135,2021,GO,Vila Boa,5222203,Secretaria Exclusiva,Feminino,Preta,Até Ensino Médio,Entre 30-49
11136,2018,GO,Vila Propício,5222302,Secretaria Exclusiva,Feminino,Branca,Até Ensino Superior Completo,Entre 30-49
11137,2021,GO,Vila Propício,5222302,Secretaria Exclusiva,Feminino,Branca,Até Ensino Superior Completo,Entre 30-49
11138,2018,DF,Brasília,5300108,Secretaria Exclusiva,Masculino,Branca,Até Pós Graduação ou Mestrado,Entre 30-49


In [67]:
df['ano'].unique()

array([2018, 2021], dtype=int64)

Subindo para o GBQ

In [68]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11140 entries, 0 to 11139
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   ano                          11140 non-null  int64   
 1   sigla_uf                     11140 non-null  object  
 2   nome_municipio               11140 non-null  object  
 3   id_municipio                 11140 non-null  int64   
 4   caracterizacao_orgao_gestor  11140 non-null  object  
 5   genero                       11140 non-null  object  
 6   cor_raca                     11140 non-null  object  
 7   grau_instrucao               11140 non-null  object  
 8   faixa_etaria                 11064 non-null  category
dtypes: category(1), int64(2), object(6)
memory usage: 794.4+ KB


In [70]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação referente municipio'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Código do IBGE do município'),
        bigquery.SchemaField('caracterizacao_orgao_gestor','STRING',description='Caracterização do órgão no qual o gestor está'),
        bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
        bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação')
        ]


In [71]:
table_ref = dataset_ref.table('MUNIC_perfil_gestor_saude_tipo_orgao')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=a724e531-35db-4782-8dd3-473304c6f610>